# Simple ASX Sector Summary Dashboard in Power BI
* Data gathering (Python)
* Data storage (SQL)
* Data access / visualisation (Power BI)

![](images/asxdashboard.png)

In [7]:
import pandas as pd
import numpy as np
import datetime
# import yfinance as yf # Doesnt have data on ASX
import pyasx.data.securities # use this instead
import pyasx.data.companies


import psycopg2 as pg

import warnings
warnings.filterwarnings('ignore')

# Get list of Single-stock derivatives from ASX

In [2]:
data = pd.read_excel('https://www2.asx.com.au/content/dam/asx/participants/derivatives-market/equity-derivatives/equity-derivatives-statistics/2022/annual-market-summary-2022.xls', sheet_name=-1)
data

,Monthly Market Summary for December 2022,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,Product Prices,NaN,Movement,Lots Traded,NaN,Traded Value,NaN,Exercises,NaN,Open Positions,NaN
1,Product,2022-12-01 00:00:00,2022-12-30 00:00:00,%,Call,Put,Call,Put,Call,Put,Call,Put
2,A2M,6.27,6.88,9.728868,13519,12857,1144923.5,725264,10214,2049,7513,9322
3,AGL,8.08,8.07,-0.123762,8970,18017,438381.5,7269974.4,6084,3540,12085,19884
4,AGL*,8.08,8.07,-0.123762,84,0,9929.5,0,0,0,651,0
...,...,...,...,...,...,...,...,...,...,...,...,...
163,XJO*,7354.4,7038.7,-4.292668,1695,4529,6062420,13451278.14,0,0,60,78
164,XJOL,7354.4,7038.7,-4.292668,1170,0,83818725,0,2111,0,230,0
165,ZIP,0.745,0.51,-31.543624,35940,25923,141647.5,1061218,2020,14565,18096,22899
166,Total,NaN,NaN,NaN,2584672,2333933,689387781.54,451969368.02,688767,438440,2959612,2228983


### Create clean list of tickers

In [3]:
df = data.iloc[2:-2,[0]]

df.rename(columns={df.columns[0]: 'ticker'}, inplace=True)

df['ticker'] = df.ticker.str.replace('*','')
df['no_L'] = df.ticker.str.replace('L', '')
df['duplicated'] = df.duplicated(subset='no_L')
df = df[df['duplicated'] == False]
df.drop(['duplicated'], axis=1, inplace=True)
df.drop(['no_L'], axis=1, inplace=True)
df['ticker'] = df['ticker'].apply(lambda x: x[:-1] if len(x) == 4 else x)

In [4]:
tickers = df.ticker.to_list()
print(len(tickers), 'Single-stock derivatives from ASX as of December 2022')
print(tickers)

89 Single-stock derivatives from ASX as of December 2022
['A2M', 'AGL', 'AKE', 'ALD', 'ALL', 'ALU', 'AMC', 'AMP', 'ANN', 'ANZ', 'APA', 'ASX', 'AWC', 'AZJ', 'BEN', 'BHP', 'BLD', 'BOQ', 'BPT', 'BSL', 'BXB', 'CBA', 'COL', 'CPU', 'CSL', 'CSR', 'CWY', 'DXS', 'EDV', 'EVN', 'FLT', 'FMG', 'GMG', 'GPT', 'HVN', 'IAG', 'IFL', 'IGO', 'ILU', 'IPL', 'JHX', 'LLC', 'LYC', 'MGR', 'MIN', 'MPL', 'MQG', 'MTS', 'NAB', 'NCM', 'NEC', 'NST', 'NXT', 'ORG', 'ORI', 'OZL', 'PBH', 'PLS', 'QAN', 'QBE', 'RHC', 'RIO', 'RRL', 'S32', 'SCG', 'SEK', 'SGM', 'SGP', 'SGR', 'SHL', 'STO', 'STW', 'SUN', 'TAH', 'TCL', 'TLS', 'TPG', 'TWE', 'VCX', 'VUK', 'WBC', 'WDS', 'WES', 'WHC', 'WOR', 'WOW', 'WTC', 'XJO', 'ZIP']


# Get security info using pyasx
https://github.com/jericmac/pyasx

In [5]:
# pyasx output for CBA
# includes PE, EPS, dividend yield, market capS, sector
res = pyasx.data.securities.get_security_info('CBA')
res['principal_activities'] = pyasx.data.companies.get_company_info('CBA')['principal_activities']
res['gics_industry'] = pyasx.data.companies.get_company_info('CBA')['gics_industry']
res['gics_sector'] = pyasx.data.companies.get_company_info('CBA')['gics_sector']
res

{'ticker': 'CBA',
 'isin': 'AU000000CBA7',
 'type': 'Ordinary Fully Paid',
 'open_price': 108.4,
 'last_price': 108.05,
 'bid_price': 108.05,
 'offer_price': 108.15,
 'last_trade_date': datetime.datetime(2023, 1, 24, 0, 0, tzinfo=tzoffset(None, 39600)),
 'day_high_price': 108.54,
 'day_low_price': 107.35,
 'day_change_price': -0.56,
 'day_change_percent': '-0.516%',
 'day_volume': 1474854,
 'prev_day_close_price': 108.61,
 'prev_day_change_percent': '-0.046%',
 'year_high_price': 109.2,
 'year_high_date': datetime.datetime(2022, 11, 25, 0, 0, tzinfo=tzoffset(None, 39600)),
 'year_low_price': 86.98,
 'year_low_date': datetime.datetime(2022, 6, 17, 0, 0, tzinfo=tzoffset(None, 36000)),
 'year_open_price': 75.27,
 'year_change_price': 32.78,
 'year_change_percent': '43.55%',
 'average_daily_volume': 2207032,
 'pe': 17.37,
 'eps': 6.254,
 'annual_dividend_yield': 3.54,
 'securities_outstanding': 1688414169,
 'market_cap': 183378662895,
 'is_suspended': False,
 'indices': [],
 'principal_act

In [11]:
info_list = []
for ticker in tickers:
    try:
        results = pyasx.data.securities.get_security_info(ticker)   
        results['principal_activities'] = pyasx.data.companies.get_company_info(ticker)['principal_activities']
        results['gics_industry'] = pyasx.data.companies.get_company_info(ticker)['gics_industry']
        results['gics_sector'] = pyasx.data.companies.get_company_info(ticker)['gics_sector']
        
        info_list.append(results)
    except:
        print(ticker, 'not available')
        pass

XJO not available


In [28]:
df = pd.DataFrame(info_list)
df = df.drop(['isin', 'type', 'indices', 'last_trade_date', 'is_suspended'], axis=1)
# df.to_csv('stock_info.csv')
df

,ticker,open_price,last_price,bid_price,offer_price,day_high_price,day_low_price,day_change_price,day_change_percent,day_volume,...,year_change_percent,average_daily_volume,pe,eps,annual_dividend_yield,securities_outstanding,market_cap,principal_activities,gics_industry,gics_sector
0,A2M,6.83,6.770,6.760,6.77,6.840,6.750,-0.07,-1.023%,1643469,...,,2951499,46.00,0.1487,0.00,733302926,5015792014,Sale of branded products in targeted markets m...,"Food, Beverage & Tobacco",Consumer Staples
1,AGL,7.63,7.690,7.690,7.70,7.720,7.580,0.07,0.919%,1556489,...,-49.778%,1995492,5.79,1.3160,3.41,672747233,5126333915,The Company operates Australia's largest retai...,Utilities,Utilities
2,AKE,13.50,13.800,13.800,13.81,13.900,13.450,0.46,3.448%,4689826,...,518.834%,4358471,17.81,0.7489,0.00,637658086,8506358867,Minerals exploration and production company wi...,Materials,Metals & Mining
3,ALD,30.42,30.530,30.520,30.54,30.760,30.085,0.48,1.597%,1069274,...,45.728%,930036,7.86,3.8230,5.36,238302099,7160978075,"Purchase, refining, distribution, and marketin...",Energy,Energy
4,ALL,33.56,34.100,34.020,34.16,34.160,33.440,0.85,2.556%,1531040,...,586.117%,1613587,23.27,1.4290,1.56,657402014,21858616966,"Designs, develops, manufactures and markets a ...",Consumer Services,Consumer Discretionary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,WHC,9.18,9.170,9.170,9.20,9.310,9.080,-0.18,-1.925%,6867089,...,436.257%,11308625,4.73,1.9760,5.13,889252161,8314507705,Coal mining and exploration,Energy,Energy
84,WOR,15.77,15.880,15.870,15.88,15.920,15.540,0.05,0.316%,1363907,...,2.518%,1177160,48.26,0.3280,3.16,525060762,8311711862,Provider of professional services to help our ...,Energy,Energy
85,WOW,34.88,34.790,34.770,34.79,35.000,34.470,-0.03,-0.086%,1339323,...,-4.945%,1908068,5.36,6.4960,2.64,1216476503,42357711834,"Food, general merchandise and specialty retail...",Food & Staples Retailing,Consumer Staples
86,WTC,55.87,56.050,56.050,56.18,56.450,55.450,0.69,1.246%,433429,...,,390735,92.73,0.5970,0.20,327033882,18104595708,Provider of software solutions to the logistic...,Software & Services,Information Technology


# Connect to psql

In [29]:
conn = pg.connect(
    host='localhost',
    user = 'postgres',
    database='fundamentals',
    password='password'
)

cur = conn.cursor()
conn.autocommit=True

### Create table with correct schema using a dtype map

In [30]:
dtype_map = {
    'float64': 'double precision',
    'int64': 'bigint',
    'bool': 'boolean',
    'datetime64': 'timestamp',
    'object': 'varchar',
}

In [31]:
columns = []
# Iterate over the columns of the DataFrame
for column_name, dtype in df.dtypes.items():
    # Get the SQL data type for the column
    sql_dtype = dtype_map.get(dtype.name)
    # Append the column and its data type to the list
    columns.append(f"{column_name} {sql_dtype}")
    
columns

['ticker varchar',
 'open_price double precision',
 'last_price double precision',
 'bid_price double precision',
 'offer_price double precision',
 'day_high_price double precision',
 'day_low_price double precision',
 'day_change_price double precision',
 'day_change_percent varchar',
 'day_volume bigint',
 'prev_day_close_price double precision',
 'prev_day_change_percent varchar',
 'year_high_price double precision',
 'year_high_date varchar',
 'year_low_price double precision',
 'year_low_date varchar',
 'year_open_price varchar',
 'year_change_price varchar',
 'year_change_percent varchar',
 'average_daily_volume bigint',
 'pe double precision',
 'eps double precision',
 'annual_dividend_yield double precision',
 'securities_outstanding bigint',
 'market_cap bigint',
 'principal_activities varchar',
 'gics_industry varchar',
 'gics_sector varchar']

In [32]:
# Drop table if exists
cur.execute('DROP TABLE IF EXISTS stock_info')

# Create table
sql_create_table = """CREATE TABLE stock_info ({}) ;""".format(', '.join(columns))
cur.execute(sql_create_table)

### Check created table columns

In [33]:
cur.execute("""SELECT column_name FROM information_schema.columns WHERE table_name = 'stock_info' """)
column_names = [row[0] for row in cur.fetchall()]
print(column_names)

['day_low_price', 'day_change_price', 'average_daily_volume', 'pe', 'eps', 'annual_dividend_yield', 'securities_outstanding', 'market_cap', 'open_price', 'day_volume', 'prev_day_close_price', 'last_price', 'year_high_price', 'bid_price', 'year_low_price', 'offer_price', 'day_high_price', 'gics_sector', 'day_change_percent', 'prev_day_change_percent', 'year_high_date', 'year_low_date', 'year_open_price', 'year_change_price', 'year_change_percent', 'principal_activities', 'gics_industry', 'ticker']


### Insert data

In [34]:
sql_insert = """INSERT INTO stock_info VALUES ({})""".format(', '.join(['%s'] * len(df.columns)))
sql_insert

'INSERT INTO stock_info VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [35]:
data = []
for index, row in df.iterrows():
    tup = tuple(row)
    data.append(tup)

In [36]:
cur.executemany(sql_insert, data)

### Query table

In [37]:
pd.read_sql("""select * from stock_info""",conn)

,ticker,open_price,last_price,bid_price,offer_price,day_high_price,day_low_price,day_change_price,day_change_percent,day_volume,...,year_change_percent,average_daily_volume,pe,eps,annual_dividend_yield,securities_outstanding,market_cap,principal_activities,gics_industry,gics_sector
0,A2M,6.83,6.770,6.760,6.77,6.840,6.750,-0.07,-1.023%,1643469,...,,2951499,46.00,0.1487,0.00,733302926,5015792014,Sale of branded products in targeted markets m...,"Food, Beverage & Tobacco",Consumer Staples
1,AGL,7.63,7.690,7.690,7.70,7.720,7.580,0.07,0.919%,1556489,...,-49.778%,1995492,5.79,1.3160,3.41,672747233,5126333915,The Company operates Australia's largest retai...,Utilities,Utilities
2,AKE,13.50,13.800,13.800,13.81,13.900,13.450,0.46,3.448%,4689826,...,518.834%,4358471,17.81,0.7489,0.00,637658086,8506358867,Minerals exploration and production company wi...,Materials,Metals & Mining
3,ALD,30.42,30.530,30.520,30.54,30.760,30.085,0.48,1.597%,1069274,...,45.728%,930036,7.86,3.8230,5.36,238302099,7160978075,"Purchase, refining, distribution, and marketin...",Energy,Energy
4,ALL,33.56,34.100,34.020,34.16,34.160,33.440,0.85,2.556%,1531040,...,586.117%,1613587,23.27,1.4290,1.56,657402014,21858616966,"Designs, develops, manufactures and markets a ...",Consumer Services,Consumer Discretionary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,WHC,9.18,9.170,9.170,9.20,9.310,9.080,-0.18,-1.925%,6867089,...,436.257%,11308625,4.73,1.9760,5.13,889252161,8314507705,Coal mining and exploration,Energy,Energy
84,WOR,15.77,15.880,15.870,15.88,15.920,15.540,0.05,0.316%,1363907,...,2.518%,1177160,48.26,0.3280,3.16,525060762,8311711862,Provider of professional services to help our ...,Energy,Energy
85,WOW,34.88,34.790,34.770,34.79,35.000,34.470,-0.03,-0.086%,1339323,...,-4.945%,1908068,5.36,6.4960,2.64,1216476503,42357711834,"Food, general merchandise and specialty retail...",Food & Staples Retailing,Consumer Staples
86,WTC,55.87,56.050,56.050,56.18,56.450,55.450,0.69,1.246%,433429,...,,390735,92.73,0.5970,0.20,327033882,18104595708,Provider of software solutions to the logistic...,Software & Services,Information Technology
